In [1]:
import pandas as pd, requests, os
from unidecode import unidecode
enaho_metadata = '../../MetadataSources/INEI/enaho_actualizado.csv'

metadata = pd.read_csv(enaho_metadata)

def clean_names(df: pd.DataFrame):
    df.columns = [unidecode(c).lower().replace(" ", "_").replace(".", "_").strip() for c in df.columns]
    df.rename(columns={'ano': 'anio', 'a_no': 'anio'}, inplace=True)

clean_names(metadata)
# metadata.rename(columns={metadata.columns[1]: 'anio'}, inplace=True)

# metadata_sample = metadata.sample(4)
# metadata_sample = metadata.query("ano == 2006")
m_s = metadata.drop(columns=['descargar', 'descargar_1', 'descargar_2', 'verpreguntas', 'verficha', 'ficha', 'encuesta', 'periodo'])
m_s.head(7)

,nro,anio,codigo_encuesta,codigo_modulo,modulo,spss,stata,csv
0,1,2024,913,1,Características de la Vivienda y del Hogar,https://proyectos.inei.gob.pe/iinei/srienaho/d...,https://proyectos.inei.gob.pe/iinei/srienaho/d...,https://proyectos.inei.gob.pe/iinei/srienaho/d...
1,2,2024,913,2,Características de los Miembros del Hogar,https://proyectos.inei.gob.pe/iinei/srienaho/d...,https://proyectos.inei.gob.pe/iinei/srienaho/d...,https://proyectos.inei.gob.pe/iinei/srienaho/d...
2,3,2024,913,3,Educación,https://proyectos.inei.gob.pe/iinei/srienaho/d...,https://proyectos.inei.gob.pe/iinei/srienaho/d...,https://proyectos.inei.gob.pe/iinei/srienaho/d...
3,4,2024,913,4,Salud,https://proyectos.inei.gob.pe/iinei/srienaho/d...,https://proyectos.inei.gob.pe/iinei/srienaho/d...,https://proyectos.inei.gob.pe/iinei/srienaho/d...
4,5,2024,913,7,Gastos en Alimentos y Bebidas (Módulo 601),https://proyectos.inei.gob.pe/iinei/srienaho/d...,https://proyectos.inei.gob.pe/iinei/srienaho/d...,https://proyectos.inei.gob.pe/iinei/srienaho/d...
5,6,2024,913,37,Programas Sociales (Miembros del Hogar),https://proyectos.inei.gob.pe/iinei/srienaho/d...,https://proyectos.inei.gob.pe/iinei/srienaho/d...,https://proyectos.inei.gob.pe/iinei/srienaho/d...
6,7,2024,913,85,"Gobernabilidad, Democracia y Transparencia",https://proyectos.inei.gob.pe/iinei/srienaho/d...,https://proyectos.inei.gob.pe/iinei/srienaho/d...,https://proyectos.inei.gob.pe/iinei/srienaho/d...


In [2]:
# print(metadata.shape)
spss_files = metadata[~(metadata['anio'].isin(metadata[['anio', 'csv']].dropna().anio.values))]
years_stata = spss_files['anio'].unique()
csv_files = metadata.query('anio not in @years_stata')

- Descargar archivo
- Unzip archivo
- Buscar todos los archivos que coincidan con la extension-
- lo que quiero hacer
    - leer cada archivo
      - como ya tengo los datos de anio y metadata
      - Obtener todas las columnas por modulo
        - esto para implementar un search
      - renombrar la columna "Año" a anio (revisar casos)
    - exportar en formato parquets los archivos en el siguiente dirtree
      - Data/INEI/ENAHO
        - data/{year}
          - {ubigeo_nn}_{mod_n}_{n}.parquet
        - metadata/{year}
          - {archivos_repetitivos}.pdf

In [3]:
import zipfile


TRASH = "../../trash/"

n = 5
anio = 2022

m_s = metadata.query('anio == @anio')

mod = m_s.codigo_modulo.values[n]

stata_url = m_s.spss.values[n]

type = 'spss'

type_file = {
    "csv": ".csv",
    'spss': ".sav",
    'pdf': '.pdf'
}


def get_extract_zip(zip_url, anio, mod_number:str):
    TRASH_ = TRASH + f"/{anio}/"
    os.makedirs(TRASH_, exist_ok=True)
    mod_number = str(mod_number).zfill(2)

    name = TRASH_ + f"{anio}_{mod_number}"
    with open(name + '.zip', 'wb') as zip_origin:
        zip_bin = requests.get(zip_url).content
        zip_origin.write(zip_bin)
    with zipfile.ZipFile(name + ".zip", 'r') as zip_ref:
        zip_ref.extractall(name)
    os.remove(name + '.zip')
    
# m_s

In [4]:
m_s.head(2)

,nro,anio,periodo,codigo_encuesta,encuesta,codigo_modulo,modulo,ficha,descargar,descargar_1,descargar_2,verpreguntas,verficha,spss,stata,csv
530,1,2022,55,784,Condiciones de Vida y Pobreza - ENAHO,1,Características de la Vivienda y del Hogar,NaN,NaN,NaN,NaN,"javascript:VerPreguntas('784','01','Condicione...","javascript:VerFicha('784','01')",https://proyectos.inei.gob.pe/iinei/srienaho/d...,https://proyectos.inei.gob.pe/iinei/srienaho/d...,https://proyectos.inei.gob.pe/iinei/srienaho/d...
531,2,2022,55,784,Condiciones de Vida y Pobreza - ENAHO,2,Características de los Miembros del Hogar,NaN,NaN,NaN,NaN,"javascript:VerPreguntas('784','02','Condicione...","javascript:VerFicha('784','02')",https://proyectos.inei.gob.pe/iinei/srienaho/d...,https://proyectos.inei.gob.pe/iinei/srienaho/d...,https://proyectos.inei.gob.pe/iinei/srienaho/d...


In [5]:
def get_all_data_year(year, data = metadata):
    df = data.copy()
    df_ref = df.query("anio == @year")
    is_spsss = year < 2006
    base_cols = ['anio', 'codigo_modulo']
    if is_spsss:
        ref_df = df_ref[base_cols + ["spss"]]
    else:
        ref_df = df_ref[base_cols + ["csv"]]
    ref_df.columns = ['anio', 'mod', 'url']
    for i, row in ref_df.iterrows():
        get_extract_zip(row['url'], row['anio'], row['mod'])

In [6]:


def search_files_ext(type = 'spss', dir_master_trash = TRASH):
    file_data = []
    for root, _, filenames in os.walk(dir_master_trash):

        for file in filenames:
            if file.endswith(type_file[type]) and "tabla" not in file.lower():  # Filtrar archivos .dta y excluir "tabla"
                file_path = os.path.join(root, file)  # Ruta absoluta
                file_size = os.path.getsize(file_path) / (1024 * 1024)  # Convertir tamaño a MB
                relative_path = os.path.relpath(root, dir_master_trash)
                first_subdirectory = relative_path.split(os.sep)[0]
                anio, mod = first_subdirectory.split('_')
                last_directory = os.path.basename(root)  # Última subcarpeta

                # Agregar datos a la lista
                file_data.append({
                    "ultima_subcarpeta": last_directory,
                    "anio": anio,
                    "mod": mod,
                    "nombre_archivo": file,
                    "peso_mb": round(file_size, 2),
                    "ruta_absoluta": file_path
                })
    return file_data
# file_data = []
# get_extract_zip(stata_url, anio, mod)
# file_data = search_files_ext()
# file_data

In [20]:
MASTER_DIR_BACKUP = '../../data/inei/enaho/'
os.makedirs(MASTER_DIR_BACKUP, exist_ok=True)


In [ ]:
import shutil
# datos de metadata de pdf sirve para referencia Probar para otro anio
year = 2022
METADATA_BACKUP = MASTER_DIR_BACKUP + f'metadata/{year}/'
os.makedirs(METADATA_BACKUP, exist_ok=True)



file_data = search_files_ext(type='pdf', dir_master_trash=TRASH + f"/{year}/")
metadata_pdf = pd.DataFrame(file_data)[['nombre_archivo', 'peso_mb']].drop_duplicates().sort_values('peso_mb')
pdf_ref = pd.DataFrame(file_data)
# pdf_ref
grouped = pdf_ref.groupby(["anio", "nombre_archivo", "peso_mb"]).agg({
    "mod": lambda x: "_".join(sorted(set(map(str, x)))),
    "ruta_absoluta": "first"  # Tomar cualquier ruta como referencia
}).reset_index()
def generate_filename(row):
    # if "_" in row["mod"]:  # Si pertenece a múltiples módulos
    return f"{row['mod']}_{row['nombre_archivo']}"
    # return row["nombre_archivo"]
# save_into = 
grouped["nombre_final"] = grouped.apply(generate_filename, axis=1)
grouped['ruta_destino'] = grouped['nombre_final'].apply(lambda x: os.path.join(METADATA_BACKUP, x))


for index, row in grouped.iterrows():
    source = row["ruta_absoluta"]  # Ruta original
    destination = row["ruta_destino"]  # Nueva ruta
    
    try:
        shutil.copy(source, destination)
        # print(f"Archivo movido: {source} -> {destination}")
        # shutil.remove(source)
    except FileNotFoundError:
        print(f"Archivo no encontrado: {source}")
    except Exception as e:
        print(f"Error al mover {source}: {e}")



# METADATA_BACKUP
# pdf_ref[['anio', 'nombre_archivo', 'peso_mb']].drop_duplicates()

In [35]:
file_data.head(1)

,ultima_subcarpeta,anio,mod,nombre_archivo,peso_mb,ruta_absoluta
0,784-Modulo01,2022,01,Enaho01-2022-100.csv,33.16,../../trash//2022/2022_01\784-Modulo01\Enaho01...


In [51]:
MASTER_DIR_BACKUP

'../../data/inei/enaho/'

In [60]:
file_data

,ultima_subcarpeta,anio,mod,nombre_archivo,peso_mb,ruta_absoluta
27,784-Modulo34,2022,34,Sumaria-2022.csv,23.38,../../trash//2022/2022_34\784-Modulo34\Sumaria...
9,784-Modulo10,2022,10,Enaho01-2022-604.csv,57.72,../../trash//2022/2022_10\784-Modulo10\Enaho01...


In [ ]:
year = 2022
data_enaho_dir = f'{MASTER_DIR_BACKUP}data/{year}'
os.makedirs(data_enaho_dir, exist_ok=True)

file_data = search_files_ext(type='csv', dir_master_trash=TRASH + f"/{year}/")
file_data = pd.DataFrame(file_data).sort_values('peso_mb')

def save_with_ubigeo(
        df: pd.DataFrame, unique_ubigeos, mod, n_mod=None, col_ref = 'n_dep'
    ):
    mod = str(mod)
    if n_mod is not None:
        mod = mod + "_" + str(n_mod)
    
    backup_data_path = []

    for ubigeo_n in unique_ubigeos:
        df_ubigeo = df[df[col_ref] == ubigeo_n]
        path_name = f"{data_enaho_dir}/mod_{mod}_ubigeo_{ubigeo_n}.parquet"
        df_ubigeo.to_parquet(path_name)
        backup_data_path.append(path_name)
    return backup_data_path
    
year_ref = {}

for i, row in file_data.iterrows():
    # print(row['mod'])
    if year < 2007:
        pass #spss
    else:
        df_mod_n = pd.read_csv(row['ruta_absoluta'], encoding='latin', low_memory=False)
    clean_names(df_mod_n)
    df_mod_n['ubigeo'] = df_mod_n['ubigeo'].apply(lambda x : str(x).zfill(6))
    df_mod_n['n_dep'] = df_mod_n['ubigeo'].apply(lambda x: x[:2])
    unique_ubigeo = df_mod_n['n_dep'].unique()

    files_names = save_with_ubigeo(df_mod_n, unique_ubigeo, row['mod'])
    year_ref[row['mod']] = dict(files_path = files_names, columns = list(df_mod_n.columns))
    del df_mod_n

In [65]:
year_ref

{'24': {'files_path': ['../../data/inei/enaho//data/2022/mod_24_ubigeo_01.parquet',
   '../../data/inei/enaho//data/2022/mod_24_ubigeo_06.parquet',
   '../../data/inei/enaho//data/2022/mod_24_ubigeo_09.parquet',
   '../../data/inei/enaho//data/2022/mod_24_ubigeo_10.parquet',
   '../../data/inei/enaho//data/2022/mod_24_ubigeo_13.parquet',
   '../../data/inei/enaho//data/2022/mod_24_ubigeo_20.parquet',
   '../../data/inei/enaho//data/2022/mod_24_ubigeo_22.parquet',
   '../../data/inei/enaho//data/2022/mod_24_ubigeo_12.parquet',
   '../../data/inei/enaho//data/2022/mod_24_ubigeo_02.parquet',
   '../../data/inei/enaho//data/2022/mod_24_ubigeo_16.parquet',
   '../../data/inei/enaho//data/2022/mod_24_ubigeo_03.parquet',
   '../../data/inei/enaho//data/2022/mod_24_ubigeo_04.parquet',
   '../../data/inei/enaho//data/2022/mod_24_ubigeo_05.parquet',
   '../../data/inei/enaho//data/2022/mod_24_ubigeo_08.parquet',
   '../../data/inei/enaho//data/2022/mod_24_ubigeo_14.parquet',
   '../../data/inei/

In [161]:
import gc


data_enaho_dir = f'../../data/inei/enaho/data/{anio}'
os.makedirs(data_enaho_dir, exist_ok=True)

ubigeo_dep = df_n1['dep_n'].unique()
ubigeo_n = ubigeo_dep[2]
backup_data = data_enaho_dir + f"./{ubigeo_n}_mod_{str(mod).zfill(2)}_1"


datetime_cols = df_n1.select_dtypes(include=['object']).columns
datetime_cols
for col in datetime_cols:
    df_n1[col] = df_n1[col].astype(str)
df_n1_filtered = df_n1[df_n1['dep_n'] == '15']

df_n1_filtered.to_csv(backup_data + '.csv', index=False)

df_csv = pd.read_csv(backup_data + '.csv')
df_csv
df_csv.to_parquet(backup_data + '.parquet')

# print(df_csv.head())

# del df_n_filtered
# del df_csv
# gc.collect()


# df_n.dtypes
# /data/inei/enaho/data/{anio}/{ubigeo}_{mod}_{v}.parquet

In [3]:
import zipfile


trash_ = "../../trash"
os.makedirs(trash_, exist_ok=True)
name = trash_ + "/file.zip"
with open(name, 'wb') as zip_b :
    zip_file = requests.get(stata_url).content
    zip_b.write(zip_file)
with zipfile.ZipFile(name, 'r') as zip_ref: 
    zip_ref.extractall(trash_)

# os.remove(name)

# requests.get(stata_url).content

In [34]:

# Recorrer directorios y subdirectorios de manera recursiva
def search_files_ext(ext = 'sav', dir_master_trash = trash_):
    file_data = []
    for root, _, filenames in os.walk(dir_master_trash):
        for file in filenames:
            if file.endswith(ext) and "tabla" not in file.lower():  # Filtrar archivos .dta y excluir "tabla"
                file_path = os.path.join(root, file)  # Ruta absoluta
                file_size = os.path.getsize(file_path) / (1024 * 1024)  # Convertir tamaño a MB
                last_directory = os.path.basename(root)  # Última subcarpeta

                # Agregar datos a la lista
                file_data.append({
                    "ultima_subcarpeta": last_directory,
                    "nombre_archivo": file,
                    "peso_mb": round(file_size, 2),
                    "ruta_absoluta": file_path
                })
    return file_data
# file_data = []
file_data = search_files_ext()

In [35]:
file_data

[{'ultima_subcarpeta': '282-Modulo05',
  'nombre_archivo': 'Enaho01a-2006-500.sav',
  'peso_mb': 46.27,
  'ruta_absoluta': '../../trash\\282-Modulo05\\Enaho01a-2006-500.sav'}]

In [37]:
pd.read_spss(file_data[0]['ruta_absoluta'], convert_categoricals=False)

,AÑO,MES,CONGLOME,VIVIENDA,HOGAR,CODPERSO,UBIGEO,DOMINIO,ESTRATO,CODINFOR,...,I559D43,I559D44,I559D45,I559D46,I560D1,I560D2,I560D3,OCU500,T560B1,FAC500A
0,2006,06,0005,009,11,01,010101,4.0,4.0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,170.41
1,2006,06,0005,009,11,02,010101,4.0,4.0,2,...,NaN,NaN,NaN,NaN,153.701233,NaN,NaN,4.0,10.0,158.42
2,2006,06,0005,009,11,03,010101,4.0,4.0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,272.40
3,2006,06,0005,009,11,04,010101,4.0,4.0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN
4,2006,06,0005,009,11,05,010101,4.0,4.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,232.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62337,2006,12,5378,107,11,01,250103,7.0,7.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,53.56
62338,2006,12,5378,107,11,02,250103,7.0,7.0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,65.42
62339,2006,12,5378,107,11,03,250103,7.0,7.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,55.92
62340,2006,12,5378,108,11,01,250103,7.0,7.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,49.97
